In [1]:
conf = {}
conf["eventhubs.connectionString"] = "Endpoint=sb://YOURSEVENTHUBNAMESPACE.servicebus.windows.net/;SharedAccessKeyName=receiver;SharedAccessKey=......;EntityPath=YOURSEVENTHUBNAME"

stream_df = (
  spark
    .readStream
    .format("eventhubs")
    .options(**conf)
    .load()
)

In [2]:
serialized_df = stream_df.select(stream_df["body"].cast("string"), stream_df["enqueuedTime"], stream_df["partition"])

query_stream = (
  serialized_df
    .writeStream
    .format("memory")
    .queryName("rawdata")
    .start()
)


In [3]:
%sql select count(*) as messages, partition  from rawdata group by partition order by partition

messages,partition
15548,0
14554,1
14127,10
15523,11
13902,12
19422,13
15454,14
14933,15
14253,16
19744,17


In [4]:
%sql select count(*) as messages, unix_timestamp(enqueuedTime) as timestamp  from rawdata group by timestamp order by timestamp 

messages,timestamp
15000,1583435813
30000,1583435814
25000,1583435815
15000,1583435816
10000,1583435817
25000,1583435818
35000,1583435819
30000,1583435820
5000,1583435821
40000,1583435822


In [5]:
from pyspark.sql import functions as F

writeConfig = {
 "Endpoint" : "https://dmcosmossink.documents.azure.com:443/",
 "Masterkey" : "sXR0AXHiZSieHDoA9anYJnqJMOiK2fjieIWPqb4FWdqJI6EZnHFg7p7Ozuw307qCyBnsN8bWd7wNL2QSOWeYKg==",
 "Database" : "dbsink",
 "Collection" : "stats",
"Upsert": "true",
    "WritingBatchSize": "500"
}

maxtransmision_df = serialized_df.withWatermark("enqueuedTime", "10 minutes").groupby(F.unix_timestamp("enqueuedTime")).count()

streamingQueryWriter = (
  maxtransmision_df
    .writeStream
    .format("com.microsoft.azure.cosmosdb.spark.streaming.CosmosDBSinkProvider")
    .outputMode("update")
    .options(**writeConfig)
    .option("checkpointLocation", "/streamingcheckpointlocation")
    .start()
)